# Deep Learning Bootcamp November 2017, GPU Computing for Data Scientists

<img src="../images/bcamp.png" align="center">

## 31-PyTorch-using-CONV1D-on-one-dimensional-data 

Web: https://www.meetup.com/Tel-Aviv-Deep-Learning-Bootcamp/events/241762893/

Notebooks: <a href="https://github.com/QuantScientist/Data-Science-PyCUDA-GPU"> On GitHub</a>

*Shlomo Kashani*

<img src="../images/pt.jpg" width="35%" align="center">


### Data
- Download from https://numer.ai/leaderboard

# PyTorch Imports


In [1]:
# !pip install pycuda
%reset -f
# %%timeit

import torch
from torch.autograd import Variable
import numpy as np
import pandas
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split
import logging
import numpy
import numpy as np
from __future__ import print_function
from __future__ import division
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MultiLabelBinarizer
import time
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
import numpy as np
import scipy
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = (6, 6)      # setting default size of plots
import tensorflow as tf 
print("tensorflow:" + tf.__version__)
!set "KERAS_BACKEND=tensorflow"
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')

# !pip install http://download.pytorch.org/whl/cu75/torch-0.2.0.post1-cp27-cp27mu-manylinux1_x86_64.whl
# !pip install torchvision 
# ! pip install cv2
# import cv2

print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)

handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)
%matplotlib inline

# !pip install psutil
import psutil
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()

# %%timeit
use_cuda = torch.cuda.is_available()
# use_cuda = False

FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

lgr.info("USE CUDA=" + str (use_cuda))

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


tensorflow:1.2.1


INFO:__main__:USE CUDA=True


__Python VERSION: 2.7.12 (default, Nov 19 2016, 06:48:10) 
[GCC 5.4.0 20160609]
__pyTorch VERSION: 0.2.0+42448cf
__CUDA VERSION
__CUDNN VERSION: None
__Number CUDA Devices: 1
__Devices
OS:  linux2
Python:  2.7.12 (default, Nov 19 2016, 06:48:10) 
[GCC 5.4.0 20160609]
PyTorch:  0.2.0+42448cf
Numpy:  1.13.1
2.7.12 (default, Nov 19 2016, 06:48:10) 
[GCC 5.4.0 20160609]
0.0
svmem(total=67469099008, available=62165921792, percent=7.9, used=4683751424, free=56004976640, active=8588935168, inactive=1747623936, buffers=1005113344, cached=5775257600, shared=71159808)
memory GB: 0.223262786865


#  Global params

In [2]:
# NN params
LR = 0.005
MOMENTUM= 0.9

# Data params
TARGET_VAR= 'target'
TOURNAMENT_DATA_CSV = 'numerai_tournament_data.csv'
TRAINING_DATA_CSV = 'numerai_training_data.csv'
BASE_FOLDER = 'numerai/'


# fix seed
seed=17*19
np.random.seed(seed)
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)

#  Train / Validation / Test Split
- Numerai provides a data set that is allready split into train, validation and test sets. 

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from collections import defaultdict

# Train, Validation, Test Split
def loadDataSplit():
    df_train = pd.read_csv(BASE_FOLDER + TRAINING_DATA_CSV)
    # TOURNAMENT_DATA_CSV has both validation and test data provided by NumerAI
    df_test_valid = pd.read_csv(BASE_FOLDER + TOURNAMENT_DATA_CSV)

    answers_1_SINGLE = df_train[TARGET_VAR]
    df_train.drop(TARGET_VAR, axis=1,inplace=True)
    df_train.drop('id', axis=1,inplace=True)
    df_train.drop('era', axis=1,inplace=True)
    df_train.drop('data_type', axis=1,inplace=True)    
            
    df_train.to_csv(BASE_FOLDER + TRAINING_DATA_CSV + 'clean.csv', header=False,  index = False)    
    df_train= pd.read_csv(BASE_FOLDER + TRAINING_DATA_CSV + 'clean.csv', header=None, dtype=np.float32)    
    df_train = pd.concat([df_train, answers_1_SINGLE], axis=1)
    feature_cols = list(df_train.columns[:-1])
    target_col = df_train.columns[-1]
    trainX, trainY = df_train[feature_cols], df_train[target_col]
    
        
    df_validation_set=df_test_valid.loc[df_test_valid['data_type'] == 'validation'] 
    df_validation_set=df_validation_set.copy(deep=True)
    answers_1_SINGLE_validation = df_validation_set[TARGET_VAR]
    df_validation_set.drop(TARGET_VAR, axis=1,inplace=True)    
    df_validation_set.drop('id', axis=1,inplace=True)
    df_validation_set.drop('era', axis=1,inplace=True)
    df_validation_set.drop('data_type', axis=1,inplace=True)
    
    
    df_validation_set.to_csv(BASE_FOLDER + TRAINING_DATA_CSV + '-validation-clean.csv', header=False,  index = False)    
    df_validation_set= pd.read_csv(BASE_FOLDER + TRAINING_DATA_CSV + '-validation-clean.csv', header=None, dtype=np.float32)    
    df_validation_set = pd.concat([df_validation_set, answers_1_SINGLE_validation], axis=1)
    feature_cols = list(df_validation_set.columns[:-1])

    target_col = df_validation_set.columns[-1]
    valX, valY = df_validation_set[feature_cols], df_validation_set[target_col]
                                
    df_test_set = pd.read_csv(BASE_FOLDER + TOURNAMENT_DATA_CSV)
    df_test_set=df_test_set.copy(deep=True)
    df_test_set.drop(TARGET_VAR, axis=1,inplace=True)
    tid_1_SINGLE = df_test_set['id']
    df_test_set.drop('id', axis=1,inplace=True)
    df_test_set.drop('era', axis=1,inplace=True)
    df_test_set.drop('data_type', axis=1,inplace=True)   
            
    feature_cols = list(df_test_set.columns) # must be run here, we dont want the ID    
    df_test_set = pd.concat([tid_1_SINGLE, df_test_set], axis=1)            
    testX = df_test_set[feature_cols].values
        
    return trainX, trainY, valX, valY, testX, df_test_set

In [4]:
trainX, trainY, valX, valY, testX, df_test_set = loadDataSplit()

print (trainX.shape)
print (trainY.shape)
print (valX.shape)
print (valY.shape)
print (testX.shape)
print (df_test_set.shape)

(108405, 21)
(108405,)
(16686, 21)
(16686,)
(45647, 21)
(45647, 22)


In [5]:
# Convert the np arrays into the correct dimention and type
# Note that BCEloss requires Float in X as well as in y
def XnumpyToTensor(x_data_np):
    x_data_np = np.array(x_data_np.values, dtype=np.float32)        
    print(x_data_np.shape)
    print(type(x_data_np))

    if use_cuda:
        lgr.info ("Using the GPU")    
        X_tensor = Variable(torch.from_numpy(x_data_np).cuda()) # Note the conversion for pytorch    
    else:
        lgr.info ("Using the CPU")
        X_tensor = Variable(torch.from_numpy(x_data_np)) # Note the conversion for pytorch
    
    print(type(X_tensor.data)) # should be 'torch.cuda.FloatTensor'            
    print((X_tensor.data.shape)) # torch.Size([108405, 29])
    return X_tensor


# Convert the np arrays into the correct dimention and type
# Note that BCEloss requires Float in X as well as in y
def YnumpyToTensor(y_data_np):    
    y_data_np=y_data_np.reshape((y_data_np.shape[0],1)) # Must be reshaped for PyTorch!
    print(y_data_np.shape)
    print(type(y_data_np))

    if use_cuda:
        lgr.info ("Using the GPU")            
    #     Y = Variable(torch.from_numpy(y_data_np).type(torch.LongTensor).cuda())
        Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor).cuda()  # BCEloss requires Float        
    else:
        lgr.info ("Using the CPU")        
    #     Y = Variable(torch.squeeze (torch.from_numpy(y_data_np).type(torch.LongTensor)))  #         
        Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor)  # BCEloss requires Float        

    print(type(Y_tensor.data)) # should be 'torch.cuda.FloatTensor'
    print(y_data_np.shape)
    print(type(y_data_np))    
    return Y_tensor

In [6]:
# import torch
# import torch
# from torch.autograd import Variable
# import torch.nn as nn
# import torch.nn.functional as F


# # Dimensions
# # Number of features for the input layer
# N_FEATURES=trainX.shape[1]
# # Number of rows
# NUM_ROWS=trainX.shape[0]
# MULT_FACTOR=32 # this number has no meaning except for being divisable by 2
# Layer1Size=N_FEATURES * MULT_FACTOR


# X_tensor_train= XnumpyToTensor(trainX) # default order is NBC for a 3d tensor, but we have a 2d tensor
# X_shape=X_tensor_train.data.size()

# fc1 = torch.nn.Sequential (
#    torch.nn.Linear (N_FEATURES, Layer1Size),
#    torch.nn.BatchNorm1d(Layer1Size, eps=1e-05, momentum=0.1, affine=True),
#    torch.nn.LeakyReLU (0.2),
# )

# lgr.info  (fc1)
# b = fc1(X_tensor_train)
# print ('(b.size():' + str (b.size())) # torch.Size([108405, 928])

# b = b.view(NUM_ROWS,N_FEATURES,MULT_FACTOR) # torch.Size([108405, 29, 32])
# print ('(b.size() after re-shaping:' + str (b.size()))

# conv1 = torch.nn.Sequential (
#    torch.nn.Conv1d(N_FEATURES, N_FEATURES * MULT_FACTOR, kernel_size=(5,), stride=(1,), padding=(1,)),
#    torch.nn.BatchNorm1d(Layer1Size, eps=1e-05, momentum=0.1, affine=True),
#    torch.nn.LeakyReLU (0.2),
# )

# lgr.info (conv1) # 

# c = conv1(b) #  Need input of dimension 4 and input.size[1] == 928 but got input to be of shape: [108405 x 29 x 1 x 32]
#              # RuntimeError: running_mean should contain 232 elements not 928
# lgr.info  (c.size() )

In [18]:
# References:
# https://github.com/vinhkhuc/PyTorch-Mini-Tutorials/blob/master/5_convolutional_net.py
# https://gist.github.com/spro/c87cc706625b8a54e604fb1024106556

X_tensor_train= XnumpyToTensor(trainX) # default order is NBC for a 3d tensor, but we have a 2d tensor
X_shape=X_tensor_train.data.size()


# Dimensions
# Number of features for the input layer
N_FEATURES=trainX.shape[1]
# Number of rows
NUM_ROWS=trainX.shape[0]
# this number has no meaning except for being divisable by 2
MULT_FACTOR=8 
# Size of first linear layer
Layer1Size=N_FEATURES * MULT_FACTOR
# CNN kernel size
CNN_KERNEL=5

dropout = torch.nn.Dropout(p=1 -.95)
sigmoid = torch.nn.Sigmoid()
tanh=torch.nn.Tanh()
relu=torch.nn.LeakyReLU()


class Net2(nn.Module):    
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net2, self).__init__()
        self.n_feature=n_feature
        
        #linear1,dropout,relu,nn.BatchNorm1d(Layer1Size)
        self.l1 = nn.Sequential(
            torch.nn.Linear(n_feature, n_hidden),
            torch.nn.LeakyReLU (0.2),
            torch.nn.Dropout(p=1 -.90),            
            torch.nn.BatchNorm1d(n_hidden, eps=1e-05, momentum=0.1, affine=True)            
        )        
        
        self.c1= nn.Sequential(
            torch.nn.Conv1d(n_feature, n_feature * MULT_FACTOR, kernel_size=(CNN_KERNEL,), stride=(1,), padding=(1,)),
            torch.nn.BatchNorm1d(n_hidden, eps=1e-05, momentum=0.1, affine=True),
            torch.nn.LeakyReLU (0.2)
        )        
        
        self.l2 = nn.Sequential(
            torch.nn.Linear(MULT_FACTOR * N_FEATURES * (MULT_FACTOR - CNN_KERNEL + 3) , n_output)
        )
        
        self.sig=nn.Sigmoid()

    def forward(self, x):
        print ('(x.size():' + str (x.size()))
        print(type(x.data)) # should be 'torch.cuda.FloatTensor'            
        x=self.l1(x)                
        print ('(x.size() after l1:' + str (x.size()))
        print(type(x.data)) # should be 'torch.cuda.FloatTensor'            
        # for CNN        
        x = x.view(NUM_ROWS,self.n_feature,MULT_FACTOR)
#         if use_cuda:
#             x=x.cuda()
        print ('(x.size() after re-shape:' + str (x.size()))
        print(type(x.data)) # should be 'torch.cuda.FloatTensor'            
        x=self.c1(x)
        print ('(x.size() after conv1d:' + str (x.size()))
        print(type(x.data)) # should be 'torch.cuda.FloatTensor'            
        # for Linear layer
        x = x.view(NUM_ROWS, self.n_feature * MULT_FACTOR * (MULT_FACTOR - CNN_KERNEL + 3))
        print ('(x.size() after re-shape 2:' + str (x.size()))
        print(type(x.data)) # should be 'torch.cuda.FloatTensor'            
        x=self.l2 (x)
        print ('(x.size() after l2:' + str (x.size()))  
        print(type(x.data)) # should be 'torch.cuda.FloatTensor'            
        x=self.sig (x)
        return x
    
net = Net2(n_feature=N_FEATURES, n_hidden=Layer1Size, n_output=1)   # define the network    
if use_cuda:
    net=net.cuda() # very important !!!
lgr.info(net)
b = net(X_tensor_train)
print ('(b.size():' + str (b.size())) # torch.Size([108405, 928])

INFO:__main__:Using the GPU
INFO:__main__:Net2 (
  (l1): Sequential (
    (0): Linear (21 -> 168)
    (1): LeakyReLU (0.2)
    (2): Dropout (p = 0.1)
    (3): BatchNorm1d(168, eps=1e-05, momentum=0.1, affine=True)
  )
  (c1): Sequential (
    (0): Conv1d(21, 168, kernel_size=(5,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(168, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.2)
  )
  (l2): Sequential (
    (0): Linear (1008 -> 1)
  )
  (sig): Sigmoid ()
)


(108405, 21)
<type 'numpy.ndarray'>
<class 'torch.cuda.FloatTensor'>
torch.Size([108405, 21])
(x.size():torch.Size([108405, 21])
<class 'torch.cuda.FloatTensor'>


RuntimeError: cuda runtime error (2) : out of memory at /home/gpu/dev/pytorch/torch/lib/THC/THCTensorCopy.cu:204

In [59]:
optimizer = torch.optim.Adam(net.parameters(), lr=LR,weight_decay=5e-4) #  L2 regularization
loss_func=torch.nn.BCELoss() # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
if use_cuda:
    lgr.info ("Using the GPU")    
    net.cuda()
    loss_func.cuda()

lgr.info (optimizer)
lgr.info (loss_func)

INFO:__main__:<torch.optim.adam.Adam object at 0x7f72b9a71390>
INFO:__main__:BCELoss (
)


In [ ]:
import time
start_time = time.time()    
epochs=300
all_losses = []

X_tensor_train= XnumpyToTensor(trainX)
Y_tensor_train= YnumpyToTensor(trainY)

print(type(X_tensor_train.data), type(Y_tensor_train.data)) # should be 'torch.cuda.FloatTensor'

# From here onwards, we must only use PyTorch Tensors
for step in range(epochs):
    out = net(X_tensor_train)                 # input x and predict based on x
    cost = loss_func(out, Y_tensor_train)     # must be (1. nn output, 2. target), the target label is NOT one-hotted

    optimizer.zero_grad()   # clear gradients for next train
    cost.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
                           
    if step % 50 == 0:        
        loss = cost.data[0]
        all_losses.append(loss)
        print(step, cost.data.cpu().numpy())        
        prediction = (net(X_tensor_train).data).float() # probabilities             
        pred_y = prediction.cpu().numpy().squeeze()
        target_y = Y_tensor_train.cpu().data.numpy()                        
        tu = ((pred_y == target_y).mean(),log_loss(target_y, pred_y),roc_auc_score(target_y,pred_y ))
        print ('ACC={}, LOG_LOSS={}, ROC_AUC={} '.format(*tu))        
                
end_time = time.time()
print ('{} {:6.3f} seconds'.format('GPU:', end_time-start_time))

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(all_losses)
plt.show()

false_positive_rate, true_positive_rate, thresholds = roc_curve(target_y,pred_y)
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('LOG_LOSS=' + str(log_loss(target_y, pred_y)))
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.6f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([-0.1, 1.2])
plt.ylim([-0.1, 1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

INFO:__main__:Using the CPU
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
INFO:__main__:Using the CPU


(108405, 21)
<type 'numpy.ndarray'>
<class 'torch.FloatTensor'>
torch.Size([108405, 21])
(108405, 1)
<type 'numpy.ndarray'>
<class 'torch.FloatTensor'>
(108405, 1)
<type 'numpy.ndarray'>
<class 'torch.FloatTensor'> <class 'torch.FloatTensor'>
